# Migrations & Schema Management

This notebook covers the full schema management lifecycle in SurrealDB-ORM: creating tables, evolving schemas through migrations, introspecting existing databases, and using advanced features like events, materialized views, and typed relations.

## Use Case: Setting Up a New Project Schema

Whether you're starting from scratch or inheriting an existing database, SurrealDB-ORM provides Django-style migrations to manage your schema as code. This ensures reproducible deployments and safe schema evolution across environments.

## Prerequisites

- SurrealDB running locally (`docker run --rm -p 8000:8000 surrealdb/surrealdb:latest start --user root --pass root`)
- Project dependencies installed (`uv sync`)
- A `.env` file in the project root (optional, falls back to defaults)

In [ ]:
# Setup: add project root to path and configure SurrealDB connection
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)

## 1. Migration System Overview

The migration system follows Django's patterns: detect model changes, generate migration files, and apply them in order.

### CLI Commands

```bash
# Generate a migration from your current model definitions
surreal-orm makemigrations --name initial

# Apply all pending migrations to the database
surreal-orm migrate

# Check which migrations have been applied
surreal-orm status

# Rollback a specific migration
surreal-orm rollback 0001_initial
```

Migrations are stored as Python files in your `migrations/` directory. Each file contains a list of operations (CreateTable, AddField, CreateIndex, etc.) that transform the schema.

## 2. Migration Operations

The migration system supports these operations:

| Operation | Description |
|---|---|
| `CreateTable` | DEFINE TABLE with optional TYPE, permissions |
| `DropTable` | REMOVE TABLE |
| `AddField` | DEFINE FIELD with type, default, assertions |
| `AlterField` | Modify an existing field |
| `RemoveField` | REMOVE FIELD |
| `CreateIndex` | DEFINE INDEX (unique, HNSW, FTS/BM25) |
| `RemoveIndex` | REMOVE INDEX |
| `DefineEvent` | DEFINE EVENT (server-side triggers) |
| `RemoveEvent` | REMOVE EVENT |
| `DefineAnalyzer` | DEFINE ANALYZER (FTS tokenizers/filters) |
| `RemoveAnalyzer` | REMOVE ANALYZER |

In [ ]:
# Example: generating SQL from migration operations
from src.surreal_orm.migrations.operations import CreateTable, AddField, CreateIndex

# CreateTable generates DEFINE TABLE
op1 = CreateTable(name="users")
print("CreateTable:", op1.forwards())

# AddField generates DEFINE FIELD
op2 = AddField(table="users", name="email", field_type="string")
print("AddField:   ", op2.forwards())

# CreateIndex generates DEFINE INDEX
op3 = CreateIndex(table="users", name="idx_email", fields=["email"], unique=True)
print("CreateIndex:", op3.forwards())

## 3. Schema Introspection

If you have an existing SurrealDB database and want to generate Python models from it, the introspection tools parse `INFO FOR DB` and `INFO FOR TABLE` results into typed model code.

This is useful when:
- Onboarding an existing database into the ORM
- Verifying that your models match the live schema
- Generating a starting point for migrations

In [ ]:
# Generate Python model code from the current database schema
from src.surreal_orm import generate_models_from_db, schema_diff

# This introspects the live database and produces Python source code
# Uncomment to run against your database:
# code = await generate_models_from_db()
# print(code)

# You can also save directly to a file:
# code = await generate_models_from_db(output_path="generated_models.py")

print("generate_models_from_db() parses INFO FOR DB/TABLE and produces Pydantic models.")
print("It handles: generic types, record links, encrypted fields, computed fields, etc.")

In [ ]:
# Schema diff: compare your Python models against the live database
# Returns a list of migration operations needed to synchronize them
from src.surreal_orm import BaseSurrealModel

class User(BaseSurrealModel):
    id: str | None = None
    name: str
    email: str

class Order(BaseSurrealModel):
    id: str | None = None
    total: float = 0.0
    status: str = "pending"

# Uncomment to run against your database:
# operations = await schema_diff(models=[User, Order])
# for op in operations:
#     print(f"  {op.__class__.__name__}: {op.forwards()}")

print("schema_diff() compares Python models vs live DB and returns needed operations.")
print("Use this to detect drift between code and database.")

## 4. DEFINE EVENT -- Server-Side Triggers

Events let you define triggers that run on the SurrealDB server whenever records are created, updated, or deleted. Think of them as database-level webhooks.

**When to use events:**
- Audit logging (track who changed what)
- Derived data updates (update a counter when a record is created)
- Notifications (create a notification record when something changes)

In [ ]:
# DefineEvent creates a server-side trigger in a migration
from src.surreal_orm.migrations.operations import DefineEvent, RemoveEvent

# This event fires when a user's email changes, creating an audit log entry
event = DefineEvent(
    name="email_audit",
    table="users",
    when="$before.email != $after.email",
    then="CREATE audit_log SET table = 'user', record = $value.id, action = $event",
)
print("DefineEvent forwards():")
print(f"  {event.forwards()}")

# RemoveEvent removes the trigger
remove = RemoveEvent(name="email_audit", table="users")
print(f"\nRemoveEvent forwards():")
print(f"  {remove.forwards()}")

In [ ]:
# Events are tracked in schema state for automatic diff detection
# When you change an event's WHEN/THEN clause, makemigrations detects it
print("Event tracking in migrations:")
print("  - Adding a new event -> generates DefineEvent")
print("  - Changing WHEN/THEN -> generates RemoveEvent + DefineEvent")
print("  - Removing an event -> generates RemoveEvent")

## 5. Materialized Views

Materialized views are read-only tables backed by a `SELECT` query. SurrealDB keeps them automatically updated when the source data changes. They're defined with `DEFINE TABLE ... AS SELECT ...`.

**When to use materialized views:**
- Pre-computed aggregations (order counts, revenue summaries)
- Denormalized read models (CQRS pattern)
- Dashboard queries that would be expensive to run on every request

In [ ]:
# Define a materialized view model
# The view_query config tells SurrealDB to keep this table in sync automatically
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict

class OrderStats(BaseSurrealModel):
    model_config = SurrealConfigDict(
        table_name="order_stats",
        view_query="SELECT status, count() AS total FROM orders GROUP BY status",
    )

    id: str | None = None
    status: str
    total: int

print(f"Table name: {OrderStats.get_table_name()}")
print(f"View query: {OrderStats.model_config.get('view_query', 'N/A')}")

In [ ]:
# Materialized views are READ-ONLY -- save/update/delete raise TypeError
# This protects you from accidentally writing to a computed table
try:
    stats = OrderStats(status="pending", total=10)
    await stats.save()
except TypeError as e:
    print(f"Expected error: {e}")
    print("Materialized views are read-only -- you can only query them.")

In [ ]:
# In migrations, CreateTable with view_query generates DEFINE TABLE ... AS (...)
op = CreateTable(
    name="order_stats",
    view_query="SELECT status, count() AS total FROM orders GROUP BY status",
)
print("Generated SQL:")
print(f"  {op.forwards()}")

## 6. TYPE RELATION -- Graph Edge Constraints

SurrealDB's graph model uses relation tables as edges between nodes. TYPE RELATION enforces which tables can appear on each side of the edge, preventing invalid graph structures.

**When to use TYPE RELATION:**
- Enforcing graph schema constraints ("only persons can like posts")
- Self-documenting edge tables
- Preventing bugs where wrong record types are connected

In [ ]:
# Define a relation (edge) table with type constraints
from src.surreal_orm.types import TableType

class Likes(BaseSurrealModel):
    model_config = SurrealConfigDict(
        table_type=TableType.RELATION,
        relation_in="person",              # Only 'person' records on the IN side
        relation_out=["blog_post", "book"], # Either 'blog_post' or 'book' on OUT
        enforced=True,                      # SurrealDB rejects violations
    )

print(f"Table type: {Likes.model_config.get('table_type')}")
print(f"IN constraint: {Likes.model_config.get('relation_in')}")
print(f"OUT constraint: {Likes.model_config.get('relation_out')}")
print(f"Enforced: {Likes.model_config.get('enforced')}")

In [ ]:
# In migrations, CreateTable generates TYPE RELATION IN ... OUT ... ENFORCED
op = CreateTable(
    name="likes",
    table_type="RELATION",
    relation_in="person",
    relation_out=["blog_post", "book"],
    enforced=True,
)
print("Generated SQL:")
print(f"  {op.forwards()}")

In [ ]:
# TableType enum values
print("Available table types:")
for tt in TableType:
    print(f"  - TableType.{tt.name} = {tt.value!r}")

## 7. Advanced Index Types in Migrations

Migrations support specialized indexes for vector search and full-text search.

In [ ]:
# HNSW vector index for similarity search
from src.surreal_orm.migrations.operations import CreateIndex, DefineAnalyzer

vec_idx = CreateIndex(
    table="documents", name="vec_idx", fields=["embedding"],
    hnsw=True, dimension=1536, dist="COSINE", vector_type="F32",
)
print("HNSW Index:")
print(f"  {vec_idx.forwards()}")

# Full-text search index with BM25 scoring
fts_idx = CreateIndex(
    table="posts", name="ft_title", fields=["title"],
    search_analyzer="english_az", bm25=(1.2, 0.75), highlights=True,
)
print("\nFTS Index:")
print(f"  {fts_idx.forwards()}")

# Custom text analyzer with tokenizers and filters
analyzer = DefineAnalyzer(
    name="english_az",
    tokenizers=["blank", "class"],
    filters=["lowercase", "snowball(english)"],
)
print("\nAnalyzer:")
print(f"  {analyzer.forwards()}")

## Summary

| Feature | Purpose | CLI Command |
|---|---|---|
| Migrations | Track schema changes as code | `makemigrations`, `migrate` |
| Introspection | Generate models from existing DB | `inspectdb` |
| Schema Diff | Detect drift between code and DB | `schemadiff` |
| Events | Server-side triggers | `DefineEvent` operation |
| Materialized Views | Auto-updated read-only tables | `CreateTable(view_query=...)` |
| TYPE RELATION | Graph edge constraints | `CreateTable(table_type="RELATION")` |

The migration system ensures your schema evolves safely across development, staging, and production environments.